### **Цель:** Реализовать класс Node (аналог Element), который умеет
### выполнять операции сложения, умножения, ReLU и вычислять градиенты
### через автоматическое дифференцирование (без использования автограда).

## Импорты

In [1]:
import unittest

## Класс Node

In [9]:
class Node:
    def __init__(self, data, _children=(), _op=''):
        self.data = float(data)
        self.grad = 0.0
        self._backward = lambda: None  # функция обратного прохода
        self._prev = set(_children)    # предыдущие узлы (граф вычислений)
        self._op = _op                 # тип операции (для отладки)

    def __repr__(self):
        return f"Node(data={self.data}, grad={self.grad})"

    # сложение
    def __add__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        out = Node(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad * 1.0
            other.grad += out.grad * 1.0
        out._backward = _backward

        return out

    # умножение
    def __mul__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        out = Node(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    # ReLU
    def relu(self):
        out = Node(self.data if self.data > 0 else 0.0, (self,), 'ReLU')

        def _backward():
            self.grad += (self.data > 0) * out.grad
        out._backward = _backward

        return out

    # backprop
    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)

        # начальный градиент для финального узла = 1
        self.grad = 1.0

        # обратный проход
        for node in reversed(topo):
            node._backward()

## Пример использования

In [10]:
a = Node(2)
b = Node(-3)
c = Node(10)
d = a + b * c
e = d.relu()
e.backward()

print(a)
print(b)
print(c)
print(d)
print(e)

Node(data=2.0, grad=0.0)
Node(data=-3.0, grad=0.0)
Node(data=10.0, grad=0.0)
Node(data=-28.0, grad=0.0)
Node(data=0.0, grad=1.0)


## Тесты (unittest)

In [14]:
class TestAutograd(unittest.TestCase):
    def test_addition(self):
        a = Node(2.0)
        b = Node(3.0)
        c = a + b
        c.backward()
        self.assertAlmostEqual(a.grad, 1.0)
        self.assertAlmostEqual(b.grad, 1.0)
        self.assertAlmostEqual(c.data, 5.0)

    def test_multiplication(self):
        a = Node(2.0)
        b = Node(3.0)
        c = a * b
        c.backward()
        self.assertAlmostEqual(a.grad, 3.0)
        self.assertAlmostEqual(b.grad, 2.0)
        self.assertAlmostEqual(c.data, 6.0)

    def test_relu_positive(self):
        a = Node(5.0)
        b = a.relu()
        b.backward()
        self.assertAlmostEqual(b.data, 5.0)
        self.assertAlmostEqual(a.grad, 1.0)

    def test_relu_negative(self):
        a = Node(-5.0)
        b = a.relu()
        b.backward()
        self.assertAlmostEqual(b.data, 0.0)
        self.assertAlmostEqual(a.grad, 0.0)

    def test_chain_negative(self):
        a = Node(2.0)
        b = Node(-3.0)
        c = Node(10.0)
        d = a + b * c
        e = d.relu()
        e.backward()
        # т.к. d = -28 < 0, ReLU обнуляет градиенты
        self.assertAlmostEqual(a.grad, 0.0)
        self.assertAlmostEqual(b.grad, 0.0)
        self.assertAlmostEqual(c.grad, 0.0)

    def test_chain_positive(self):
        a = Node(2.0)
        b = Node(3.0)
        c = Node(10.0)
        d = a + b * c  # d = 32 > 0
        e = d.relu()
        e.backward()
        # ReLU активна, поэтому градиенты не обнулены
        self.assertAlmostEqual(a.grad, 1.0)
        self.assertAlmostEqual(b.grad, 10.0)
        self.assertAlmostEqual(c.grad, 3.0)


if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.006s

OK
